In [1]:
!pip install scipy
!pip install numpy
!pip install pandas

In [2]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
import implicit

c:\python38\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
train_path = r"C:\Users\dlbol\Downloads\likes\likes\likes_data\train"
test_path = r"C:\Users\dlbol\Downloads\likes\likes\likes_data\test"
target_val_path = r"C:\Users\dlbol\Downloads\likes\likes\likes_data\target_val"
pred_val_path = r"C:\Users\dlbol\Downloads\likes\likes\likes_data\pred_val"

In [8]:
def calc_mrr(predict, answer):
    for i in range(len(predict)):
        if predict[i] == answer:
            return 1. / (i + 1)
    return 0

max_prediction_len = 100

def calc_score(target_path, predict_path):
    with open(target_path) as f:
        y_true = [int(x.strip()) for x in f.readlines()]

    with open(predict_path) as f:
        y_pred = [[int(x) for x in line.strip().split(' ')] for line in f.readlines()]

    mrr_score = 0
    for (pred, answer) in zip(y_pred, y_true):
        if len(pred) > max_prediction_len:
            raise ValueError('$maximum prediction length is {}, got {}$'.format(max_prediction_len, len(y_pred[i])))
        mrr_score += calc_mrr(pred, answer)

    print(f"MRR@100 = {(mrr_score / len(y_true)):.4f}")

In [9]:
%%time

list_coord = []
list_coord_val = []
target_val = []
np.random.seed(42)  
val_id = np.random.choice(range(10001), size=1000, replace=False)
val_id.sort()

with open(train_path) as f:
    lines = f.readlines()
    idx = 0
    idx_val = 0
    for line in lines:
        tracks = line.strip().split(' ')
        if len(tracks) == 0:
            print(0)
        if len(tracks) == 1:
            print(1)
        for i, track in enumerate(tracks):
            if idx in val_id and i+1 == len(tracks):
                target_val.append(track)
                idx_val += 1
                continue
            if idx in val_id:
                list_coord_val.append((idx_val, track))
            list_coord.append((idx, track))
        idx += 1
        if idx >= 10000:
            break

CPU times: total: 4 s
Wall time: 4.02 s


In [10]:
%%time

arr_rows = np.array([i[0] for i in list_coord])
arr_cols = np.array([i[1] for i in list_coord]).astype(int)
ones = np.ones(arr_rows.shape[0])
user_item = csr_matrix((ones, (arr_rows, arr_cols)), dtype=int)
#item_user = csr_matrix((ones, (arr_cols, arr_rows)), dtype=int)

arr_rows_val = np.array([i[0] for i in list_coord_val])
arr_cols_val = np.array([i[1] for i in list_coord_val]).astype(int)
ones_val = np.ones(arr_rows_val.shape[0])
user_item_val = csr_matrix((ones_val, (arr_rows_val, arr_cols_val)), dtype=int)

CPU times: total: 375 ms
Wall time: 376 ms


In [ ]:
import time

def optimize(factors=100,
             regularization=0.01,
             alpha=1.0,
             use_native=True,
             use_cg=True,
             iterations=15,
             calculate_training_loss=False):
    
    model = implicit.als.AlternatingLeastSquares(factors=factors, 
                                                 regularization=regularization,
                                                 alpha=alpha,
                                                 use_native=use_native,
                                                 use_cg=use_cg,
                                                 iterations=iterations,
                                                 random_state=42)
 
    model.fit(user_item)
    
    start_time = time.time()
    
    userid = val_id
    recommendations = model.recommend(userid, user_item_val, N=100, filter_already_liked_items=True)
    result = [' '.join(map(str, i)) + '\n' for i in recommendations[0]]
    
    print("--- %s seconds ---" % (time.time() - start_time))
    
    with open(pred_val_path, 'w') as f:
        f.writelines(result)
    
    with open(target_val_path, 'w') as f:
        for i in target_val:
            f.write(f'{i}\n')
            
    calc_score(target_val_path, pred_val_path)

In [239]:
for use_cg in [False, True]:
    for factors in [5, 10, 15, 30, 50, 100]:
        for iterations in [5, 10, 15, 30, 50, 100]:
            for alpha in [0.001, 0.01, 0.1, 1, 10, 100]:
                for regularization in [0.001, 0.01, 0.1, 1, 10]:
                    print(f'use_cg, {use_cg}, factors: {factors}, iterations: {iterations}, alpha: {alpha}, regularization: {regularization}')
                    optimize(factors=factors,
                         regularization=regularization,
                         alpha=alpha,
                         use_native=True,
                         use_cg=True,
                         iterations=iterations,
                         calculate_training_loss=False)

use_cg, False, factors: 5, iterations: 5, alpha: 0.001, regularization: 0.001


  0%|          | 0/5 [00:00<?, ?it/s]

--- 0.9567289352416992 seconds ---
MRR@100 = 0.0048
use_cg, False, factors: 5, iterations: 5, alpha: 0.001, regularization: 0.01


  0%|          | 0/5 [00:00<?, ?it/s]

--- 0.959949254989624 seconds ---
MRR@100 = 0.0052
use_cg, False, factors: 5, iterations: 5, alpha: 0.001, regularization: 0.1


...

--- 1.4372987747192383 seconds ---
MRR@100 = 0.0018
use_cg, False, factors: 30, iterations: 10, alpha: 0.01, regularization: 10


  0%|          | 0/10 [00:00<?, ?it/s]

--- 1.30472993850708 seconds ---
MRR@100 = 0.0005
use_cg, False, factors: 30, iterations: 10, alpha: 0.1, regularization: 0.001


  0%|          | 0/10 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
for use_cg in [False, True]:
    for factors in [30, 50, 100]:
        for iterations in [30, 50, 100]:
                print(f'use_cg, {use_cg}, factors: {factors}, iterations: {iterations}')
                optimize(factors=factors,
                     regularization=10,
                     alpha=10,
                     use_native=True,
                     use_cg=True,
                     iterations=iterations,
                     calculate_training_loss=False)

use_cg, False, factors: 30, iterations: 30


  0%|          | 0/30 [00:00<?, ?it/s]

--- 1.1639349460601807 seconds ---
MRR@100 = 0.0142
use_cg, False, factors: 30, iterations: 50


  0%|          | 0/50 [00:00<?, ?it/s]

--- 1.3995869159698486 seconds ---
MRR@100 = 0.0138
use_cg, False, factors: 30, iterations: 100


...

  0%|          | 0/30 [00:00<?, ?it/s]

--- 1.208704948425293 seconds ---
MRR@100 = 0.0169
use_cg, True, factors: 50, iterations: 50


  0%|          | 0/50 [00:00<?, ?it/s]

--- 1.314753770828247 seconds ---
MRR@100 = 0.0177
use_cg, True, factors: 50, iterations: 100


  0%|          | 0/100 [00:00<?, ?it/s]

In [12]:
for factors in [100, 150, 200]:
    for iterations in [20, 30, 40]:
            print(f'factors: {factors}, iterations: {iterations}')
            optimize(factors=factors,
                 regularization=10,
                 alpha=10,
                 use_native=True,
                 use_cg=False,
                 iterations=iterations,
                 calculate_training_loss=False)

factors: 100, iterations: 20


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:20<00:00,  1.04s/it]


--- 0.7612245082855225 seconds ---
MRR@100 = 0.0208
factors: 100, iterations: 30


100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:31<00:00,  1.05s/it]


--- 0.7656757831573486 seconds ---
MRR@100 = 0.0207
factors: 100, iterations: 40


...

--- 1.0113377571105957 seconds ---
MRR@100 = 0.0218
factors: 150, iterations: 30


100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [15:38<00:00, 31.27s/it]


--- 1.0153064727783203 seconds ---
MRR@100 = 0.0216
factors: 150, iterations: 40


 10%|████████▎                                                                          | 4/40 [02:35<23:16, 38.80s/it]


KeyboardInterrupt: 

In [21]:
for factors in [100, 150, 200]:
    for iterations in [15, 20, 25]:
            print(f'factors: {factors}, iterations: {iterations}')
            optimize(factors=factors,
                 regularization=10,
                 alpha=10,
                 use_native=True,
                 use_cg=False,
                 iterations=iterations,
                 calculate_training_loss=False)
 

factors: 100, iterations: 15


 27%|██████████████████████▏                                                            | 4/15 [00:22<01:01,  5.62s/it]


KeyboardInterrupt: 

In [14]:
for factors in [100, 150, 200]:
        print(f'factors: {factors}')
        optimize(factors=factors,
             regularization=10,
             alpha=10,
             use_native=True,
             use_cg=False,
             iterations=20,
             calculate_training_loss=False)

factors: 100


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:21<00:00,  1.05s/it]


--- 0.7596590518951416 seconds ---
MRR@100 = 0.0208
factors: 150


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [26:42<00:00, 80.13s/it]


--- 0.9017114639282227 seconds ---
MRR@100 = 0.0218
factors: 200


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [14:06<00:00, 42.35s/it]


--- 1.0894722938537598 seconds ---
MRR@100 = 0.0227


In [22]:
%%time

list_coord = []
list_coord_val = []
target_val = []
np.random.seed(42)  
val_id = np.random.choice(range(100000), size=25000, replace=False)
val_id.sort()

with open(train_path) as f:
    lines = f.readlines()
    idx = 0
    idx_val = 0
    for line in lines:
        tracks = line.strip().split(' ')
        if len(tracks) == 0:
            print(0)
        if len(tracks) == 1:
            print(1)
        for i, track in enumerate(tracks):
            if idx in val_id and i+1 == len(tracks):
                target_val.append(track)
                idx_val += 1
                continue
            if idx in val_id:
                list_coord_val.append((idx_val, track))
            list_coord.append((idx, track))
        idx += 1
        if idx >= 100000:
            break

CPU times: total: 2min 8s
Wall time: 2min 8s


In [23]:
%%time

arr_rows = np.array([i[0] for i in list_coord])
arr_cols = np.array([i[1] for i in list_coord]).astype(int)
ones = np.ones(arr_rows.shape[0])
user_item = csr_matrix((ones, (arr_rows, arr_cols)), dtype=int)
#item_user = csr_matrix((ones, (arr_cols, arr_rows)), dtype=int)

arr_rows_val = np.array([i[0] for i in list_coord_val])
arr_cols_val = np.array([i[1] for i in list_coord_val]).astype(int)
ones_val = np.ones(arr_rows_val.shape[0])
user_item_val = csr_matrix((ones_val, (arr_rows_val, arr_cols_val)), dtype=int)

CPU times: total: 3.61 s
Wall time: 3.61 s


In [24]:
import time

def optimize(factors=100,
             regularization=0.01,
             alpha=1.0,
             use_native=True,
             use_cg=True,
             iterations=15,
             calculate_training_loss=False):
    
    model = implicit.als.AlternatingLeastSquares(factors=factors, 
                                                 regularization=regularization,
                                                 alpha=alpha,
                                                 use_native=use_native,
                                                 use_cg=use_cg,
                                                 iterations=iterations,
                                                 random_state=42)
 
    model.fit(user_item)
    
    start_time = time.time()
    
    userid = val_id
    recommendations = model.recommend(userid, user_item_val, N=100, filter_already_liked_items=True)
    result = [' '.join(map(str, i)) + '\n' for i in recommendations[0]]
    
    print("--- %s seconds ---" % (time.time() - start_time))
    
    with open(pred_val_path, 'w') as f:
        f.writelines(result)
    
    with open(target_val_path, 'w') as f:
        for i in target_val:
            f.write(f'{i}\n')
            
    calc_score(target_val_path, pred_val_path)

In [25]:
for factors in [50, 100, 150]:
        print(f'factors: {factors}')
        optimize(factors=factors,
             regularization=10,
             alpha=10,
             use_native=True,
             use_cg=False,
             iterations=20,
             calculate_training_loss=False)

factors: 50


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:31<00:00,  1.59s/it]


--- 14.807275533676147 seconds ---
MRR@100 = 0.0172
factors: 100


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [01:39<00:00,  4.99s/it]


--- 17.90154457092285 seconds ---
MRR@100 = 0.0206
factors: 150


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [30:57<00:00, 92.87s/it]


--- 26.543158054351807 seconds ---
MRR@100 = 0.0219


In [26]:
for use_cg in [False, True]:
        print(f'use_cg: {use_cg}')
        optimize(factors=100,
             regularization=10,
             alpha=10,
             use_native=True,
             use_cg=use_cg,
             iterations=20,
             calculate_training_loss=False)

use_cg: False


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [02:03<00:00,  6.15s/it]


--- 22.88296937942505 seconds ---
MRR@100 = 0.0206
use_cg: True


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [11:12<00:00, 33.64s/it]


--- 17.10799503326416 seconds ---
MRR@100 = 0.0206


In [27]:
for iterations in [10, 20, 50, 100]:
        print(f'iterations: {iterations}')
        optimize(factors=100,
             regularization=10,
             alpha=10,
             use_native=True,
             use_cg=False,
             iterations=iterations,
             calculate_training_loss=False)

iterations: 10


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:46<00:00,  4.63s/it]


--- 16.07869529724121 seconds ---
MRR@100 = 0.0204
iterations: 20


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [01:36<00:00,  4.84s/it]


--- 16.63031530380249 seconds ---
MRR@100 = 0.0206
iterations: 50


100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [04:06<00:00,  4.93s/it]


--- 16.422136783599854 seconds ---
MRR@100 = 0.0205
iterations: 100


  2%|█▋                                                                                | 2/100 [00:14<12:01,  7.36s/it]


KeyboardInterrupt: 

In [ ]:
for alpha in [0.001, 0.01, 0.1, 1, 10, 100]:
    for regularization in [0.001, 0.01, 0.1, 1, 10]:
        print(f' alpha: {alpha}, regularization: {regularization}')
        optimize(factors=100,
             regularization=regularization,
             alpha=alpha,
             use_native=True,
             use_cg=False,
             iterations=20,
             calculate_training_loss=False)

 alpha: 0.001, regularization: 0.001


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [01:37<00:00,  4.89s/it]


--- 16.729506731033325 seconds ---
MRR@100 = 0.0077
 alpha: 0.001, regularization: 0.01


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [01:39<00:00,  4.97s/it]


--- 16.79504680633545 seconds ---
MRR@100 = 0.0088
 alpha: 0.001, regularization: 0.1


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [01:37<00:00,  4.90s/it]


--- 16.690091609954834 seconds ---
MRR@100 = 0.0083
 alpha: 0.001, regularization: 1


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [17:16<00:00, 51.82s/it]


In [ ]:
for alpha in [0.1, 1, 10, 100]:
    for regularization in [0.1, 1, 10, 100]:
        print(f' alpha: {alpha}, regularization: {regularization}')
        optimize(factors=100,
             regularization=regularization,
             alpha=alpha,
             use_native=True,
             use_cg=False,
             iterations=20,
             calculate_training_loss=False)